Oznaczenia (predykaty i stałe):

  Loves(x,y): x kocha y

  Kills(x,y): x zabija y

  Animal(y): y jest zwierzęciem

  Tuna: kot Tuna (stała)

  Jack, Jola: osoby (stałe)

  ∀, ∃: kwantyfikatory ogólny i szczegółowy
    
Wiedza ogólna (KB):
1. Każdy, kto kocha wszystkie zwierzęta, jest przez kogoś kochany.
    ∀x[(∀y(Animal(y)→Loves(x,y)))→(∃zLoves(z,x))]
2. Nikt nie kocha tego, kto zabija zwierzę.
    ∀x∀y∀z[(Animal(z)∧Kills(y,z))→¬Loves(x,y)]
3. Jack kocha wszystkie zwierzęta.
    ∀y(Animal(y)→Loves(Jack,y))
4. Albo Jack, albo Jola zabili kota Tuna.
    Kills(Jack,Tuna)∨Kills(Jola,Tuna)
5. Tuna jest zwierzęciem.
    Animal(Tuna)



In [6]:
from sympy import symbols, Or, And, Not
from sympy.logic.boolalg import to_cnf
from sympy.logic.inference import satisfiable

# Predykaty jako funkcje
Loves = lambda x, y: symbols(f"Loves_{x}_{y}")
Kills = lambda x, y: symbols(f"Kills_{x}_{y}")
Animal = lambda x: symbols(f"Animal_{x}")

# Stałe (osoby i kot)
Jack = "Jack"
Jola = "Jola"
Tuna = "Tuna"

# KB: baza wiedzy w postaci CNF, po ręcznej skolemizacji

kb = And(
    # (1) Jack kocha wszystkie zwierzęta → ¬Animal(Tuna) ∨ Loves(Jack, Tuna)
    Or(Not(Animal(Tuna)), Loves(Jack, Tuna)),

    # (2) Tuna jest zwierzęciem
    Animal(Tuna),

    # (3) Jack lub Jola zabili kota Tuna
    Or(Kills(Jack, Tuna), Kills(Jola, Tuna)),

    # (4) Nikt nie kocha tego, kto zabija zwierzę → rozpisane dla Jacka i Joli
    Or(Not(Animal(Tuna)), Not(Kills(Jack, Tuna)), Not(Loves("Someone", Jack))),
    Or(Not(Animal(Tuna)), Not(Kills(Jola, Tuna)), Not(Loves("Someone", Jola))),

    # (5) Każdy, kto kocha wszystkie zwierzęta, jest przez kogoś kochany
    # Rozpisane dla Jacka i Tuna (skolemizowane)
    Or(Animal(Tuna), Loves("f", Jack)),
    Or(Not(Loves(Jack, Tuna)), Loves("f", Jack))
)

# Negacja wniosku: zakładamy, że Jola NIE zabiła Tuny
negated_conclusion = Not(Kills(Jola, Tuna))

# Łączymy bazę wiedzy z negacją wniosku
full_formula = And(kb, negated_conclusion)

# Przekształcamy do CNF (można też wypisać: print(to_cnf(full_formula, simplify=True)))
cnf = to_cnf(full_formula, simplify=True)

# Sprawdzamy, czy formuła jest spełnialna
result = satisfiable(cnf)

# Interpretacja wyniku
if not result:
    print("Wniosek: Z KB wynika, że Jola zabiła kota Tuna.")
else:
    print("Nie można jednoznacznie stwierdzić, że Jola zabiła kota Tuna.")
    print("Spełniające przypisanie:", result)


Nie można jednoznacznie stwierdzić, że Jola zabiła kota Tuna.
Spełniające przypisanie: {Loves_f_Jack: True, Animal_Tuna: True, Kills_Jack_Tuna: True, Loves_Jack_Tuna: True, Kills_Jola_Tuna: False, Loves_Someone_Jack: False}
